In [1]:
#######################################################################
# This script was written by Nana Owusu, it is meant to preprocess    #
# metabolomic information from CSV files and using the mean absolute  #
# deviation of each treatment group, remove outliers.                 #
#######################################################################
# Modules for text interpretation and math
import os, sys, re, fnmatch
import numpy as np
# Modules for plotting and reading csv
# import matplotlib.pyplot as plt
# from matplotlib.figure import Figure
# import csv
import pandas as pd
# %matplotlib inline
# from ipywidgets import interactive
# Module for GUI
# import tkinter as tk
# from tkinter import filedialog
# Module for saving plots as PDF
# from matplotlib.backends.backend_pdf import PdfPages

## Load csv files with pandas

In [2]:
dataFile = pd.read_csv("/Users/nowusu/uiHackyHour/uihh_nowusu/20190723_chronicrotenoneisradipine_driftnorm.csv")

###     Group metabolite columns by drug treatment

In [3]:
## variable that contains the names of each metabolite measured
# The syntax here is known as a "list comprehension"
columns = [col for col in dataFile.columns if col not in ["Condition", "sample"]]

# gives a Pandas Series of the 21 treatment drugs
groups = dataFile['Condition']

### Get metabolite values sorted into a multi-index format

In [4]:
# Routine for numbering each individual sample
# treated with a particular drug. The final
# output is a list of tuples (a python data type defined by ())
drug_enumerate = [(groups[0],0)]
count = 0
oldDrug = groups[0]
for i in range(1,len(groups)):
    if oldDrug == groups[i]:
        count += 1
    else :
        count = 0
    drug_enumerate.append((groups[i],count))
    oldDrug = groups[i]


drug_multiIdx = pd.MultiIndex.from_tuples(drug_enumerate)

# the multi-index Pandas object will be added to this dataframe object
# for allow for better group analyses using pandas tools for stats.
metabolites_mi = pd.DataFrame(dataFile[columns])
metabolites_mi.set_axis(labels=drug_multiIdx, axis='index', inplace=True)

In [5]:
metabolites_mi

X1.Octadecanol  X2.Hydroxybutyrate  \
Isradipine          0        1.040873            1.131138   
                    1        1.077633            1.056377   
                    2        0.972901            0.995689   
                    3        1.120797            0.971689   
                    4        1.189482            1.179786   
Isradipine+Rotenone 0        1.051199            1.061541   
                    1        0.951072            1.119360   
                    2        0.985426            1.027503   
                    3        1.023643            1.031288   
Rotenone            0        1.180639            0.953011   
                    1        1.008503            1.095886   
                    2        0.944983            1.128522   
                    3        0.875994            1.005659   
                    4        0.934887            0.955510   
                    5        0.962582            1.226802   
Vehicle             0        0.990781            1.018448   
                    1        0.969146            1.063302   
                    2        0.957327            1.098904   
                    3        1.013684            1.054757   
                    4        0.918264            1.010184   
                    5        0.933773            1.054565   

                       X2.Hydroxyglutarate  X2.Oxoadipate  X3.Hydroxypyruvate  \
Isradipine          0             1.062883       0.959504            1.059183   
                    1             1.463635       1.439655            0.780903   
                    2             1.385253       1.383549            0.362198   
                    3             0.973966       1.136060            1.945651   
                    4             0.983789       1.133995            1.191188   
Isradipine+Rotenone 0             1.543269       1.332296            1.157584   
                    1             1.046026       0.948473            1.242748   
                    2             1.296233       1.326930            0.972275   
                    3             1.156100       1.137521            1.581426   
Rotenone            0             0.437137       0.625078            0.820780   
                    1             0.776436       0.659496            0.760426   
                    2             0.858093       0.821937            0.926124   
                    3             0.956380       0.955579            1.029088   
                    4             1.336476       1.250059            0.838309   
                    5             0.921649       0.803768            0.536962   
Vehicle             0             1.093740       1.003387            1.339430   
                    1             1.273897       1.157104            1.163876   
                    2             1.043531       1.008887            1.639998   
                    3             1.252248       1.285249            1.169168   
                    4             1.050831       1.040321            1.145809   
                    5             0.917510       0.899699            1.134608   

                       X3.Phosphoglycerate  X6.Phosphogluconate  Aconitate  \
Isradipine          0             1.442928             0.937136   1.400339   
                    1             1.327279             1.529427   1.311075   
                    2             1.148379             1.419673   1.231772   
                    3             0.850187             1.211664   1.230800   
                    4             0.475886             1.245228   1.280423   
Isradipine+Rotenone 0             1.108452             1.595933   1.550948   
                    1             0.777225             0.996103   0.923527   
                    2             1.227894             1.419509   1.249656   
                    3             0.645601             1.152196   1.238969   
Rotenone            0             0.652504             0.601315   1.362454   
                    1             1.092

## Functions for calculating group statistics

In [6]:
def getConditions(condGroups):
    ''' Routine for counting how many constituents are in a sequence
    after the first occurrence and saves the constituent as well
    as the count '''
    
    drugs = {}
    for treatment in condGroups:
        if treatment not in drugs:
            drugs[treatment] = 0
        drugs[treatment] += 1
    
    return drugs

def stdErr(group,metabSet):
    # calculate standard deviation for
    # each group    
    metabStdErr = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].std(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabStdErr.set_axis(axis='index', labels=group, inplace=True)
    metabStdErr.columns.names = ['Standard Deviation']
    return metabStdErr
    

def meanStdErr(group,metabSet):
    # calculate mean standard error 
    # of each group    
    metabMeanStdErr = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].sem(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabMeanStdErr.set_axis(axis='index', labels=group, inplace=True)
    metabMeanStdErr.columns.names = ['Mean Std. Error']
    
    return metabMeanStdErr

def coefOfVar(group,metabStdErr,metabMean):
    # calculate coefficient of variation 
    # of each group    
    metabCoefOfVar = metabStdErr.truediv(other=metabMean,axis='index')
    
    metabCoefOfVar.columns.names = ['Coeff. of Variation']
    
    return metabCoefOfVar

def mean(group,metabSet):
    # calculate mean of each group    
    metabMean = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].mean(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabMean.set_axis(axis='index', labels=group, inplace=True)
    metabMean.columns.names = ['Mean Std. Deviation']
    
    return metabMean

def grubbs(group,metabSet,metabMean,metabStdErr):
    # perform Grubb's analysis

    meanAbsDev = pd.DataFrame([])
    for treatment in group:
            operand = metabSet.loc[treatment,columns].sub \
                        (metabMean.loc[treatment,columns])
            operand = operand.abs()
            meanAbsDev = meanAbsDev.append(operand.div(metabStdErr.loc[treatment,columns]))
    
    meanAbsDev.set_axis(labels=drug_multiIdx,axis='index',inplace=True)
    meanAbsDev.columns.names = ['Mean Abs. Deviation']
    
    return meanAbsDev

## Calculate standard deviations

In [7]:
conditions = getConditions(groups)
std = stdErr(conditions,metabolites_mi)
std

## Calculate Averages

In [8]:
avg = mean(conditions,metabolites_mi)
avg

## Calculate Coefficient of Variation

In [9]:
cv = coefOfVar(conditions,std,avg)
cv

## Perform Grubb's Analysis

In [10]:
madVals = grubbs(conditions,metabolites_mi,avg,std)
madVals

## Function for determining outliers

In [11]:
def outliers(grubbsData,initVals,thresh):
    
    # If the condition above is true, replace the value with NaN
    testDF = grubbsData.gt(thresh)
    valueCheck = grubbsData.mask(cond=testDF,other=np.nan)
    
    # if condition above is false, replace with previous value
    valueCheck = valueCheck.where(cond=testDF,other=initVals)
    
    return valueCheck

In [12]:
whichVals = outliers(madVals,metabolites_mi,1.15)
whichVals

Mean Abs. Deviation    X1.Octadecanol  X2.Hydroxybutyrate  \
Isradipine          0        1.040873            1.131138   
                    1        1.077633            1.056377   
                    2             NaN            0.995689   
                    3        1.120797            0.971689   
                    4             NaN                 NaN   
Isradipine+Rotenone 0        1.051199            1.061541   
                    1             NaN                 NaN   
                    2        0.985426            1.027503   
                    3        1.023643            1.031288   
Rotenone            0             NaN            0.953011   
                    1        1.008503            1.095886   
                    2        0.944983            1.128522   
                    3        0.875994            1.005659   
                    4        0.934887            0.955510   
                    5        0.962582                 NaN   
Vehicle             0        0.990781            1.018448   
                    1        0.969146            1.063302   
                    2        0.957327                 NaN   
                    3             NaN            1.054757   
                    4             NaN                 NaN   
                    5        0.933773            1.054565   

Mean Abs. Deviation    X2.Hydroxyglutarate  X2.Oxoadipate  X3.Hydroxypyruvate  \
Isradipine          0             1.062883            NaN            1.059183   
                    1                  NaN            NaN            0.780903   
                    2             1.385253       1.383549                 NaN   
                    3             0.973966       1.136060                 NaN   
                    4             0.983789       1.133995            1.191188   
Isradipine+Rotenone 0                  NaN       1.332296            1.157584   
                    1             1.046026            NaN            1.242748   
                    2             1.296233       1.326930            0.972275   
                    3             1.156100       1.137521                 NaN   
Rotenone            0                  NaN       0.625078            0.820780   
                    1             0.776436       0.659496            0.760426   
                    2             0.858093       0.821937            0.926124   
                    3             0.956380       0.955579                 NaN   
                    4                  NaN            NaN            0.838309   
                    5             0.921649       0.803768                 NaN   
Vehicle             0             1.093740       1.003387            1.339430   
                    1                  NaN       1.157104            1.163876   
                    2             1.043531       1.008887                 NaN   
                    3             1.252248            NaN            1.169168   
                    4             1.050831       1.040321            1.145809   
                    5                  NaN            NaN            1.134608   

Mean Abs. Deviation    X3.Phosphoglycerate  X6.Phosphogluconate  Aconitate  \
Isradipine          0             1.442928                  NaN        NaN   
                    1             1.327279                  NaN   1.311075   
                    2             1.148379             1.419673   1.231772   
                    3             0.850187             1.211664   1.230800   
                    4                  NaN             1.245228   1.280423   
Isradipine+Rotenone 0             1.108452             1.595933        NaN   
                    1             0.777225             0.996103        NaN   
                    2             1.227894             1.419509   1.249656   
                    3             0.645601             1.152196   1.238969   
Rotenone            0                  NaN                  NaN   1.362454   
                